In [20]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train_master.tsv",sep="\t")

In [21]:
import os
import zipfile
import torch
from torchvision import datasets, models, transforms
from PIL import Image
import re

In [22]:
# transformsの定義
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}
image_datasets = {
    'train': datasets.ImageFolder('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train /train',  data_transforms['train']),
    'val': datasets.ImageFolder('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train /val', data_transforms['val'])
}

# dataloaders 作成
image_dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=20, shuffle=True, num_workers=0, drop_last=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=20, shuffle=False, num_workers=0, drop_last=True),
}
import torch.nn as nn
import torch.optim as optim
device = "cpu"
TARGET_NUM = 4

# モデル作成関数の定義
def get_model(target_num,isPretrained=False):
    """for get model"""

    model_ft = models.resnet18(pretrained=isPretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)
    return model_ft

# モデルのインスタンス作成
model = get_model(TARGET_NUM,isPretrained=False)
# 最適化関数定義
optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)

# loss関数定義
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
from torch.utils.data import Dataset
from PIL import Image, ImageFilter

In [28]:
import re
TARGET_NUM = 4
test = pd.DataFrame(data= os.listdir('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/test'))
test = test.rename(columns={0: 'filename'})
test['filename'] = sorted(test['filename'], key=lambda x: int(re.search(r'(\d+)', x).group()))
# targetカラム作成
test['target'] = 0

# 事前学習済みResNet18モデルのインスタンス作成
pretrained_model = get_model(target_num=TARGET_NUM)
pretrained_model.load_state_dict(torch.load('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/original_model_0.pth', map_location=lambda storage, loc: storage), strict=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [41]:
# 推論用のtransforms作成
test_transforms = transforms.Compose([
               transforms.Resize(256),
               transforms.ToTensor(),
               transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),

])

# dataset作成
class Test_Datasets(Dataset):

    def __init__(self, data_transform):

        self.df = test
        self.data_transform = test_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        file = self.df['filename'][index]
        image = Image.open('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/test/'+ file)
        image = image.convert('RGB')
        image = self.data_transform(image)
        return image,file


In [42]:
# datasetのインスタンス作成
test_dataset = Test_Datasets(data_transform=test_transforms)

# dataloader作成
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=1,
                                                shuffle=False,
                                                num_workers=0 ,
                                                drop_last=True )

In [43]:
pred = []  # 予測を保存するリストを初期化
for i,(inputs, labels) in enumerate(test_dataloader):
    inputs = inputs.to(device)
    # 学習済みモデルを推論モードに設定
    pretrained_model.eval()
    # 学習済みモデルにデータをインプットし、推論をさせる
    outputs = pretrained_model(inputs)
    print(outputs)
    _, preds_idx = torch.max(outputs,1)

    # インデックスを0〜3の値に変換
    preds = preds_idx.tolist()

    # 事前に用意したリストに推論結果を格納
    pred.extend(preds)


tensor([[ 0.2056,  0.3218, -0.0603,  0.2637]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3025,  0.2696, -0.1294,  0.2187]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2150,  0.2716, -0.1861,  0.2621]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2874,  0.2505, -0.1264,  0.2211]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3007,  0.1864, -0.0985,  0.1434]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2548,  0.2285, -0.0977,  0.2637]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2922,  0.3051, -0.1076,  0.2439]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3203,  0.2199, -0.1260,  0.2302]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2555,  0.1965, -0.0528,  0.3176]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2680,  0.2129, -0.0825,  0.2462]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3061,  0.2502, -0.0110,  0.2681]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3285,  0.3070, -0.1360,  0.3270]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3064,  0.2204, -0.1257,  0.2395]], grad_fn=<AddmmBackward0>)
tensor([[ 0.3002,  0.2789, -0.1391,  0.3834]], grad

In [44]:
ans = []
for i in pred:
    if i == 0:
        ans.append('angry')
    elif i == 1:
        ans.append('happy')
    elif i == 2:
        ans.append('neutral')
    else:
        ans.append('sad')

test['target'] = ans
print(ans)
test.to_csv('/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/test.csv',  header=False, index=False)

['happy', 'angry', 'happy', 'angry', 'angry', 'sad', 'happy', 'angry', 'sad', 'angry', 'angry', 'angry', 'angry', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'angry', 'sad', 'angry', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'sad', 'happy', 'sad', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'happy', 'sad', 'angry', 'happy', 'sad', 'angry', 'angry', 'sad', 'angry', 'sad', 'sad', 'happy', 'sad', 'angry', 'sad', 'angry', 'angry', 'sad', 'angry', 'sad', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'sad', 'angry', 'angry', 'sad', 'angry', 'angry', 'angry', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'sad', 'angry', 'angry', 'sad', 'sad', 'happy', 'angry', 'angry', 'angry', 'angry', 'sad', 'angry', 'sad', 'angry', 'happy', 'angry', 'sad', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'angry', 'sad', 'sad', 'sad', 'angry', 'angry', 'angry', 'angry', 'happy', 'angry', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'ang

In [45]:
root_path='/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train /val'
for id, expression in zip(df['id'], df['expression']):
      # 新しいフォルダーパスを生成
      new_folder_path = f"{root_path}/{expression}/{id}"
      # 旧フォルダーパスを生成
      old_folder_path = f"{root_path}/{id}"
      if int(re.findall(r'\d+', id)[0]) > 250:
      # フォルダーを移動する
         os.replace(old_folder_path, new_folder_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train /val/train_0251.jpg' -> '/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/train /val/angry/train_0251.jpg'

In [ ]:

# モデル学習用関数
def train_model(model, criterion, optimizer, num_epochs=5,is_saved = False):
    best_acc = 0.0

    # エポック数だけ下記工程の繰り返し
    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            print('{}:フェイズ'.format(phase))

            # 訓練フェイズと検証フェイズの切り替え
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # dataloadersからバッチサイズだけデータ取り出し、下記工程（1−5）の繰り返し
            for i,(inputs, labels) in enumerate(image_dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 1. optimizerの勾配初期化
                optimizer.zero_grad()

                # 2.モデルに入力データをinputし、outputを取り出す
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                print(preds)
                # 3. outputと正解ラベルから、lossを算出
                loss = criterion(outputs, labels)
                print('   loaders:{}回目'.format(i+1)  ,'   loss:{}'.format(loss))

                if phase == 'train':
                    # 4. 誤差逆伝播法により勾配の算出
                    loss.backward()
                    # 5. optimizerのパラメータ更新
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            dataset_sizes = len(image_datasets[phase])
            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # C. 今までのエポックでの精度よりも高い場合はモデルの保存
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                if(is_saved):
                    torch.save(model.state_dict(), '/content/drive/MyDrive/signate/練習 モノクロ顔画像の感情抽出/original_model_{}.pth'.format(epoch))

    print('Best val Acc: {:4f}'.format(best_acc))
train_model(model, criterion, optimizer, num_epochs=5,is_saved =True)